In [1]:
# importation
import numpy as np
from torch.autograd import Variable
import torch
import matplotlib.pyplot as plt
%matplotlib inline
# local
from FBResNet.myfunc import Physics
from FBResNet.myfunc import MyMatmul
from FBResNet.main import FBRestNet
from FBResNet.model import MyModel
from FBResNet.posttreat import Export_hyper

In [2]:
exp1 = FBRestNet(experimentation=Physics(2000,50,1,1), constraint = 'slab',\
                 nb_blocks=20, noise = 0.05,\
                 folder = './', im_set="Set1",batch_size=[30,1],\
                 lr=1e-3, nb_epochs=[10,1])

In [3]:
train_set, val_set = exp1.CreateDataSet()

In [4]:
exp1.train(train_set,val_set,save_model=True)

RuntimeError: shape '[1, 1, 1]' is invalid for input of size 50

### Gaussian

In [ ]:
exp1.test_gauss(noise=0.01)

### Testing set

In [ ]:
avrg_loss = exp1.test(train_set)

### Lipschitz

In [ ]:
exp1.model.Lipschitz(opt1="semi",opt2="entree11")

In [ ]:
exp1.model.Lipschitz(opt1="semi",opt2="entree01")